In [8]:
#讀入向量空間模型，給定任一篇文件，以kNN演算法自動判斷類別
#Big Data & Business Analytics, Willie Yang, National Taiwan University

In [1]:
from scipy import sparse
X = sparse.load_npz("limit_model.npz") #讀入稀疏矩陣
X

<7393x108 sparse matrix of type '<class 'numpy.float64'>'
	with 173482 stored elements in Compressed Sparse Row format>

In [3]:
#以下套用之前範例讀入詞彙集
import csv

termset=dict() #用字典預備儲存所有詞彙
no=0 #幫詞彙編號

f = open('limit_up_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

f = open('limit_down_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

print("done")

done


In [4]:
#給定任何一段內容當作查詢
query='PC品牌大廠華碩（2357）今（16）日舉行線上法說會，受惠顯示卡及PC需求暢旺，\
  帶動華碩去年品牌毛利率、營益率、淨利率「三率三升」，稅後純益大增68%，創下歷史新高，\
  全年每股純益（EPS）達60元，大賺近6個股本。華碩今日公布去年品牌財報，毛利率20.3%，\
  年增3.1個百分點。營益率9.6%，年增3個百分點。稅後純益445.5億元，年增68%。\
  淨利率8.9%，年增2個百分點。全年每股純益達60元'

#以下套用斷詞小範例
import monpa
from monpa import utils
str='' #暫存本篇斷詞後的內容用
sentence_list = utils.short_sentence(query) #斷句
for item in sentence_list:
  result_cut = monpa.cut(item) #斷詞
  for term in result_cut:
    term=term.strip() #去除前後多餘空白
    if(len(term)>1): #若詞長>1
      str=str+' '+term
print(str)

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.
 PC 品牌 大廠 華碩 2357 16 舉行 法說會 顯示卡 PC 需求 暢旺 帶動 華碩 去年 品牌 毛利率 營益率 淨利率 純益 大增 68 創下 歷史 純益 EPS 60 股本 華碩 今日 公布 去年 品牌 財報 毛利率 20.3 年增 3.1 百分點 營益率 9.6 年增 百分點 純益 445.5 年增 68 淨利率 8.9 年增 百分點 純益 60


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=termset,use_idf=True) #用給定的termset建構向量，並使用idf加權 (這邊作法值得商榷)
q=vectorizer.fit_transform([str]) #將查詢也套入同一個向量空間
print(q[0:1]) #印出該向量做觀察

  (0, 101)	0.31622776601683794
  (0, 77)	0.31622776601683794
  (0, 76)	0.31622776601683794
  (0, 64)	0.31622776601683794
  (0, 62)	0.31622776601683794
  (0, 58)	0.6324555320336759
  (0, 34)	0.31622776601683794


/Users/andy/opt/anaconda3/envs/monpa-env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [6]:
#以下套用之前範例計算相似文件
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(q[0:1], X).flatten() #與給定文件集的向量做相似度計算
related_docs_indices = cosine_similarities.argsort() #將相似度由小至大做排序，並轉換成文件編號
d=related_docs_indices[:-8:-1] #從後面取7個文件編號 (也就是相似度最大的前7名)
print(d)

[2147 4158   27 7127  870 4863  968]


In [7]:
#以kNN相似文件來投票決定類別
pred_up=0
pred_down=0
for i in d:
  if(i>=4195): #已知文件編號0~4194為漲停，4195~7393為跌停，故依此範圍判定 (此為取巧權宜作法)
    pred_down+=1 #票數加1
  else:
    pred_up+=1 #票數加1
print('pred_up:', pred_up, 'pred_down:', pred_down)

if(pred_up>pred_down):
  print('pred_label: up') #判定為漲停
elif(pred_up<pred_down):
  print('pred_label: down') #判定為跌停
else:
  print('pred_label: even') #票數平手 
    

pred_up: 5 pred_down: 2
pred_label: up
